<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#WSGI" data-toc-modified-id="WSGI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>WSGI</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# WSGI

**WSGI (pronounced wiz-gee) stands for [Web Server Gateway Interface](https://en.wikipedia.org/wiki/Web_Server_Gateway_Interface)**. It was created as an implementation-agnostic interface between web servers and web applications or frameworks to promote common ground for portable web application development in Python. Without this standard, it can become confusing or troublesome as the developers' choice of web framework will limit their choice of usable web servers, and vice versa.

The same idea can be seen in the Java world. Java has many web application frameworks available, but Java's "servlet" API makes it possible for applications written with any Java web application framework to run in any web server that supports the servlet API.

We can read the spec here in [PEP 333](https://www.python.org/dev/peps/pep-0333/), Long story short, it boils down to the concept of the python app making a callable object (e.g. a function) available to the web server. When the web server receives a request from the client that should be processed by the python app, it calls this function. The python app runs and returns the result to the web server, and the server passes it on to the client.

In the Python world, **Flask is a popular lightweight framework for writing WSGI compliant web applications in Python**. The Flask framework is implemented using the Werkzeug library, which is a library that consists of a large set of utilities designed to make writing WSGI compliant Python applications easier.

**[uWSGI](https://uwsgi-docs.readthedocs.io/en/latest/index.html) is a popular web server that implements the WSGI standard**. Don't get confused by the name. WSGI is a specification, uWSGI is a web server. That little "u" in the front makes a big difference. **It's pretty common to pair Flask and uWSGI since they both talk WSGI**.

The uWSGI server is a full fledged HTTP server that is quite capable of running production web apps. However, it's not as performant as nginx (pronounced engine-x) at serving static content, so if we need super high throughput for static content, then we have the option of sticking nginx in front of our uWSGI server. This gives us the following data flow from the client to the python app: 

<img src="img/python_app_flow.png" height="60%" width="60%">

Here nginx is served as what is called a proxy server. Now, you might be asking Why can't nginx directly call my Flask application? Because nginx doesn't support the WSGI spec. Technically nginx could implement the WSGI spec if they wanted, they just haven't. That being the case, we need a web server that does implement the spec, which is what the uWSGI server is for.

When we do put nginx in front of our uWSGI server, they will communicate over a low level protocol known as **uwsgi**. This might be a bit confusing as the naming isn't that great. When we reference uWSGI you are talking about an http server. When we say uwsgi (all lowercase) we are referring to a binary protocol that the uWSGI server uses to talk to other servers like nginx.

So to recap, the point of WSGI is to allow any web server to work with any Python application as long as both sides implement the spec. In other words, WSGI is a contract between Python applications (Flask, Django, etc) and web servers (UWSGI, Gunicorn, etc). The benefit is that we can change web servers with little effort because we know they comply with the WSGI specification and these servers are the one that are responsible for loading our Python web application using the WSGI interface.

In [1]:
import requests

show = "south park"
urlshow = "%20".join(show.split(" "))

base = 'http://api.tvmaze.com/singlesearch/shows?q='
request_info = requests.get(base + urlshow)
request_info

<Response [200]>

In [4]:
import json

json.loads(request_info.text)

{'id': 112,
 'url': 'http://www.tvmaze.com/shows/112/south-park',
 'name': 'South Park',
 'type': 'Animation',
 'language': 'English',
 'genres': ['Comedy'],
 'status': 'Running',
 'runtime': 30,
 'premiered': '1997-08-13',
 'officialSite': 'http://southpark.cc.com',
 'schedule': {'time': '22:00', 'days': ['Wednesday']},
 'rating': {'average': 8.7},
 'weight': 99,
 'network': {'id': 23,
  'name': 'Comedy Central',
  'country': {'name': 'United States',
   'code': 'US',
   'timezone': 'America/New_York'}},
 'webChannel': None,
 'externals': {'tvrage': 5266, 'thetvdb': 75897, 'imdb': 'tt0121955'},
 'image': {'medium': 'http://static.tvmaze.com/uploads/images/medium_portrait/0/935.jpg',
  'original': 'http://static.tvmaze.com/uploads/images/original_untouched/0/935.jpg'},
 'summary': '<p><b>South Park</b> is an adult comedy animation show centred around 4 children in the small town of south park. Its humour is often dark involving satirical elements and mocking current real-life events.</

# Reference

- [Stackoverflow: What is the point of uWSGI?](https://stackoverflow.com/questions/38601440/what-is-the-point-of-uwsgi)
- [Blog: An introduction into the WSGI ecosystem](http://www.ultravioletsoftware.com/single-post/2017/03/23/An-introduction-into-the-WSGI-ecosystem)
- [Blog: WSGI Servers](https://www.fullstackpython.com/wsgi-servers.html)